In [2]:
!pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/5c/98/d145af334fd5807d6ba1ead447bf0c57a36654ea58e726d70c0d09cae913/tensorflow-2.19.0-cp312-cp312-win_amd64.whl.metadata
  Using cached tensorflow-2.19.0-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
  Obtaining dependency information for keras>=3.5.0 from https://files.pythonhosted.org/packages/3f/0a/678ebcf4b6dad6ad63dfc2445d190f79a97fa7bc7150f57a6c505459e2bc/keras-3.9.2-py3-none-any.whl.metadata
  Using cached keras-3.9.2-py3-none-any.whl.metadata (6.1 kB)
  Obtaining dependency information for rich from https://files.pythonhosted.org/packages/0d/9b/63f4c7ebc259242c89b3acafdb37b41d1185c07ff0011164674e9076b491/rich-14.0.0-py3-none-any.whl.metadata
  Using cached rich-14.0.0-py3-none-any.whl.metadata (18 kB)
Using cached tensorflow-2.19.0-cp312-cp312-win_amd64.whl (376.0 MB)
Using cached keras-3.9.2-py3-none-any.whl (1.3 MB)
Using cached rich-14.0.0-py3-none-any.whl (243 kB)


ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python312\\Scripts\\import_pb_to_tensorboard.exe' -> 'C:\\Python312\\Scripts\\import_pb_to_tensorboard.exe.deleteme'


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from sklearn.utils import class_weight
from sklearn.metrics import roc_auc_score


# Compute class weights
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(class_weights))

In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import roc_auc_score

# Load your dataset
df = pd.read_csv("event_model_dataset.csv")

# Select relevant features
feature_columns = [
    'price_rating_encoded', 'expected_likes', 'expected_checkins', 'expected_rating',
    'city_encoded', 'event_weekday_num', 'is_holiday'
] + [f'type_tag_{i}' for i in range(50) if f'type_tag_{i}' in df.columns]

X = df[feature_columns]
y = df['success_flag']

# Impute missing values
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Build the neural network
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary output
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.002), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=64, validation_split=0.2, verbose=1, class_weight=class_weights)
auc = roc_auc_score(y_test, y_pred_prob)
print(f"AUC: {auc:.4f}")
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# Predict on test set
y_pred_prob = model.predict(X_test).flatten()
y_pred = (y_pred_prob > 0.05).astype(int)

# Print classification report and confusion matrix
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


# Assume y_test and y_pred are defined
report = classification_report(y_test, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()

# Optional: round and clean up
df_report = df_report.round(6)
df_report.reset_index(inplace=True)
df_report.rename(columns={'index': 'label'}, inplace=True)

# Display
print(df_report)


Epoch 1/30


c:\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.9555 - loss: 0.2871 - val_accuracy: 0.9658 - val_loss: 0.2134
Epoch 2/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9677 - loss: 0.2095 - val_accuracy: 0.9658 - val_loss: 0.1732
Epoch 3/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9658 - loss: 0.1761 - val_accuracy: 0.9658 - val_loss: 0.1713
Epoch 4/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9503 - loss: 0.2143 - val_accuracy: 0.9658 - val_loss: 0.1755
Epoch 5/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9675 - loss: 0.1336 - val_accuracy: 0.9658 - val_loss: 0.1683
Epoch 6/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9691 - loss: 0.1228 - val_accuracy: 0.9658 - val_loss: 0.1639
Epoch 7/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9765 - loss: 0.0912 - val_accuracy: 0.9658 - val_loss: 0.1617
Epoch 8/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9835 - loss: 0.0813 - val_accuracy: 0.9658 - val_loss: 0.1635
Epoch 9/30
8/8 ━